<a href="https://colab.research.google.com/github/Adithya280399/Adithya_INFO5731_Fall2025/blob/main/Suresh_Adithya_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [16]:
# Install required library (just in case)
!pip install requests pandas

import requests
import pandas as pd
import time

# Semantic Scholar API endpoint
BASE_URL = "https://api.semanticscholar.org/graph/v1/paper/search"

# Search queries
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]

# Parameters
limit = 100       # maximum per request
max_results = 10000   # total abstracts we want
results = []

for query in queries:
    offset = 0
    while len(results) < max_results:
        params = {
            "query": query,
            "fields": "title,abstract,authors,year,url",
            "limit": limit,
            "offset": offset
        }
        response = requests.get(BASE_URL, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code}, stopping.")
            break

        data = response.json()
        papers = data.get("data", [])

        if not papers:
            break

        for paper in papers:
            results.append({
                "title": paper.get("title"),
                "abstract": paper.get("abstract"),
                "authors": ", ".join([a["name"] for a in paper.get("authors", [])]),
                "year": paper.get("year"),
                "url": paper.get("url")
            })

        offset += limit
        time.sleep(0.2)  # avoid rate limits

        if len(results) >= max_results:
            break

# Convert to DataFrame
df = pd.DataFrame(results[:max_results])

# Save CSV
df.to_csv("semantic_scholar_papers.csv", index=False, encoding="utf-8")
print(f"✅ Saved {len(df)} abstracts to semantic_scholar_papers.csv")

# Preview the first 5 rows
df.head()


Error: 429, stopping.
Error: 429, stopping.
Error: 429, stopping.
Error: 429, stopping.
✅ Saved 0 abstracts to semantic_scholar_papers.csv


""


In [9]:
from google.colab import files
files.download("semantic_scholar_papers.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [10]:
!pip install pandas nltk

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources (first time only)
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("omw-1.4")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
# Load the CSV you created in Q1
df = pd.read_csv("semantic_scholar_papers.csv")

print("✅ Loaded dataset with", len(df), "rows")
df.head(3)   # preview first few


✅ Loaded dataset with 1000 rows


,title,abstract,authors,year,url
0,Fashion-MNIST: a Novel Image Dataset for Bench...,"We present Fashion-MNIST, a new dataset compri...","Han Xiao, Kashif Rasul, Roland Vollgraf",2017.0,https://www.semanticscholar.org/paper/f9c602cc...
1,Physics-informed machine learning,NaN,"G. Karniadakis, I. Kevrekidis, Lu Lu, P. Perdi...",2021.0,https://www.semanticscholar.org/paper/53c9f3c3...
2,TensorFlow: Large-Scale Machine Learning on He...,TensorFlow is an interface for expressing mach...,"Martín Abadi, Ashish Agarwal, P. Barham, E. Br...",2016.0,https://www.semanticscholar.org/paper/9c9d7247...


In [12]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # (1) Remove noise (special characters & punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)

    # (2) Remove numbers
    text = re.sub(r"\d+", " ", text)

    # (3) Lowercase
    text = text.lower()

    # Tokenize
    tokens = nltk.word_tokenize(text)

    # (4) Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # (5) Stemming
    stemmed = [stemmer.stem(word) for word in tokens]

    # (6) Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]

    return " ".join(lemmatized)


In [13]:
# Apply cleaning to the 'abstract' column
df["cleaned_abstract"] = df["abstract"].apply(clean_text)

print("✅ Cleaning done. New column 'cleaned_abstract' added.")
df[["abstract", "cleaned_abstract"]].head(5)


✅ Cleaning done. New column 'cleaned_abstract' added.


,abstract,cleaned_abstract
0,"We present Fashion-MNIST, a new dataset compri...",present fashion mnist new dataset compris x gr...
1,NaN,
2,TensorFlow is an interface for expressing mach...,tensorflow interfac express machin learn algor...
3,With the widespread use of artificial intellig...,widespread use artifici intellig ai system app...
4,NaN,


In [14]:
df.to_csv("semantic_scholar_papers_cleaned.csv", index=False, encoding="utf-8")
print("✅ Saved cleaned dataset to semantic_scholar_papers_cleaned.csv")

from google.colab import files
files.download("semantic_scholar_papers_cleaned.csv")


✅ Saved cleaned dataset to semantic_scholar_papers_cleaned.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[**Semantics Cleaned data Available here**](https://myunt-my.sharepoint.com/:x:/g/personal/adithyasuresh2_my_unt_edu/EQCYxZvjJ0ZLkSLj1ndpyukBkEn4m5JCAnGn5iX-BhLRIw?e=8VaKnu)

# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [15]:
# Install required libraries
!pip install spacy benepar

# Download spacy English model and benepar model
import spacy
import benepar

spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# Download benepar model
benepar.download('benepar_en3')

# Add benepar for constituency parsing
if spacy.__version__.startswith("3"):
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})

# Load your cleaned data
import pandas as pd

df = pd.read_csv("semantic_scholar_papers_cleaned.csv")

# Pick one column (cleaned_abstract) for analysis
texts = df["cleaned_abstract"].dropna().head(5).tolist()  # just first 5 to demo
print("✅ Loaded texts for analysis")

# -------- (1) POS Tagging -------- #
from collections import Counter

pos_counts = Counter()
for doc in nlp.pipe(texts):
    for token in doc:
        if token.pos_ in ["NOUN", "VERB", "ADJ", "ADV"]:
            pos_counts[token.pos_] += 1

print("\n🔹 POS Tagging Results:")
print(pos_counts)


# -------- (2) Constituency & Dependency Parsing -------- #
example_text = texts[0]  # take first abstract as example
doc = nlp(example_text)

print("\n🔹 Constituency Parse Tree:")
for sent in doc.sents:
    print(sent._.parse_string)   # benepar parse tree

print("\n🔹 Dependency Parsing:")
for token in doc:
    print(f"{token.text:<15} {token.dep_:<10} {token.head.text:<15} {token.pos_}")

print("\n✅ Example shown for one sentence above.")


# -------- (3) Named Entity Recognition -------- #
entity_counts = Counter()
for doc in nlp.pipe(texts):
    for ent in doc.ents:
        entity_counts[ent.label_] += 1

print("\n🔹 Named Entity Counts:")
print(entity_counts)

# Show example entities
print("\n🔹 Example Entities from first text:")
for ent in doc.ents:
    print(f"{ent.text:<20} -> {ent.label_}")


  Preparing metadata (setup.py) ... done
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37625 sha256=dba21ee77dbfa8d31117919825bf18f69e489f96734da9ecdcf7a1eff56705e9
  Stored in directory: /root/.cache/pip/wheels/9b/84/c1/f2ac877f519e2864e7dfe52a1c17fe5cdd50819cb8d1f1945f
Successfully built benepar
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


✅ Loaded texts for analysis


/usr/local/lib/python3.12/dist-packages/torch/distributions/distribution.py:62: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(



🔹 POS Tagging Results:
Counter({'NOUN': 194, 'VERB': 63, 'ADJ': 28, 'ADV': 7})

🔹 Constituency Parse Tree:
(S (NP (NP (JJ present) (NN fashion) (FW mnist)) (NP (JJ new) (NN dataset))) (VBN compris) (NP (FW x) (JJ grayscal) (NN imag) (NN fashion) (NN product) (CD categori) (NN imag) (IN per) (NP (JJ categori) (NN train) (NN set) (NN imag) (NN test) (NN set)) (FW imag) (NN fashion) (FW mnist) (VB intend) (FW serv) (NP (NML (NP (JJ direct) (NN drop)) (NN replac) (NN origin) (FW mnist)) (NN dataset) (NN benchmark) (NN machin) (VB learn) (NP (NN algorithm)) (VB share) (NN imag) (NN size) (NNS data) (NN format) (VB structur) (NN train) (NN test) ('' split) (NN dataset) (RB freeli) (VP (VB avail) (NP (FW http) (NN url))))))

🔹 Dependency Parsing:
present         amod       mnist           ADJ
fashion         compound   mnist           NOUN
mnist           nmod       grayscal        NOUN
new             amod       grayscal        ADJ
dataset         nmod       grayscal        PROPN
compris   

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [17]:
!pip install requests beautifulsoup4 pandas nltk spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import requests, csv, time, random, re, html, json, logging
from bs4 import BeautifulSoup
import pandas as pd

# NLP imports
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
#Part 1
BASE_URL = "https://github.com/marketplace"
LISTING_PATH_TEMPLATE = BASE_URL + "?type=actions&page={page}"

def get_page_html(page_num, session, retries=3):
    url = LISTING_PATH_TEMPLATE.format(page=page_num)
    for attempt in range(retries):
        try:
            r = session.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=15)
            if r.status_code == 200:
                return r.text
            time.sleep(2)
        except Exception as e:
            print(f"Error on page {page_num}: {e}")
            time.sleep(2)
    return None


def parse_page(html, page_number):
    soup = BeautifulSoup(html, "html.parser")
    results = []

    # Try to find name links using the specific observed class
    name_links = soup.select("a.marketplace-common-module__marketplace-item-link--AeQSq")
    # Fallback: any anchor within marketplace/actions link
    if not name_links:
        name_links = soup.select("a[href*='/marketplace/actions/']")

    for link in name_links:
        name = link.get_text(" ", strip=True)

        href = link.get("href", "")
        url = "https://github.com" + href if href.startswith("/") else href

        # Try to get the description with the observed <p> class
        desc_tag = link.find_next("p", class_="mt-2 mb-auto height-full text-small fgColor-muted line-clamp-2")
        # Fallback: find next <p> with any text
        if not desc_tag:
            # possibly the <p> is not nearby with that exact class
            desc_tag = link.find_next("p")

        description = desc_tag.get_text(" ", strip=True) if desc_tag else ""

        results.append({
            "product_name": name,
            "description": description,
            "url": url,
            "page_number": page_number
        })

    return results

def scrape_marketplace(start_page=1, end_page=5):
    session = requests.Session()
    all_items = []
    for p in range(start_page, end_page+1):
        print(f"Fetching page {p}...")
        html = get_page_html(p, session)
        if not html:
            continue
        items = parse_page(html, p)
        all_items.extend(items)
        time.sleep(1 + random.random())
    return pd.DataFrame(all_items)

# Example: scrape first 3 pages (adjust for ~1000 products)
df_raw = scrape_marketplace(1, 3)
df_raw.to_csv("github_marketplace_raw.csv", index=False)
df_raw.head()


Fetching page 1...
Fetching page 2...
Fetching page 3...


,product_name,description,url,page_number
0,TruffleHog OSS,Find and verify leaked credentials in your sou...,https://github.com/marketplace/actions/truffle...,1
1,Metrics embed,An infographics generator with 40+ plugins and...,https://github.com/marketplace/actions/metrics...,1
2,yq - portable yaml processor,"create, read, update, delete, merge, validate ...",https://github.com/marketplace/actions/yq-port...,1
3,Super-Linter,Super-linter is a ready-to-run collection of l...,https://github.com/marketplace/actions/super-l...,1
4,Gosec Security Checker,Runs the gosec security checker,https://github.com/marketplace/actions/gosec-s...,1


In [20]:
# Safe copy
df = df_raw.copy()

print("Columns in df_raw:", df.columns.tolist())

# Ensure required columns exist
for col in ["product_name", "description", "url"]:
    if col not in df.columns:
        df[col] = ""

STOPWORDS = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if pd.isna(text): return ""
    text = html.unescape(str(text))
    text = re.sub(r"<[^>]+>", " ", text)       # remove HTML tags
    text = re.sub(r"https?://\S+", " ", text)  # remove URLs
    text = re.sub(r"[^\w\s]", " ", text)       # remove special chars
    return re.sub(r"\s+", " ", text).strip()

def tokenize_lemmatize(text):
    if not text: return ""
    doc = nlp(text.lower())
    tokens = [t.lemma_ for t in doc if not t.is_stop and t.is_alpha]
    return " ".join(tokens)

# Apply cleaning safely
df["product_name"] = df["product_name"].astype(str).apply(clean_text)
df["description"] = df["description"].astype(str).apply(clean_text)
df["description_tokens"] = df["description"].apply(tokenize_lemmatize)

# Data Quality Report
report = {
    "rows": len(df),
    "missing_names": int(df["product_name"].eq("").sum()),
    "missing_desc": int(df["description"].eq("").sum()),
    "duplicate_urls": int(df.duplicated(subset=["url"]).sum()),
    "avg_desc_length": float(df["description"].str.len().mean())
}

df.to_csv("github_marketplace_cleaned.csv", index=False)
with open("data_quality_report.json","w") as f:
    json.dump(report, f, indent=2)

print("✅ Cleaning done. Data & report saved.")
report


Columns in df_raw: ['product_name', 'description', 'url', 'page_number']
✅ Cleaning done. Data & report saved.


{'rows': 60,
 'missing_names': 0,
 'missing_desc': 0,
 'duplicate_urls': 0,
 'avg_desc_length': 64.53333333333333}

In [21]:
from google.colab import files
files.download("github_marketplace_raw.csv")
files.download("github_marketplace_cleaned.csv")
files.download("data_quality_report.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[**Raw and cleaned data of github market place available here**](https://myunt-my.sharepoint.com/:f:/g/personal/adithyasuresh2_my_unt_edu/EnamMABWs4tNgphdbDz04k8BG7Wtyvms7Qg9QznkVa0U0g?e=1VblE8)

#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [10]:
# ==============================================================
# PART 0: Setup
# ==============================================================

!pip install tweepy pandas nltk

import tweepy
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
# ==============================================================
# PART 1: Collect Tweets
# ==============================================================

# Replace with your keys from the Canvas tutorial
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAJHs4QEAAAAAaUmVpLpVqGmv2NWu9xO9RJlrYgI%3DMX5QnDqZIQ2StDcNNM7L7Z3CTEYJXVd4BqRjpi415eCIZHIrYs"

# Initialize Tweepy client
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define query: hashtags + remove retweets + English only
query = "#machinelearning OR #artificialintelligence -is:retweet lang:en"

# Fetch up to 100 tweets in a single request
tweets = client.search_recent_tweets(
    query=query,
    tweet_fields=["id", "text", "author_id", "created_at", "lang"],
    max_results=20
)

# Convert to DataFrame
data = []
for tweet in tweets.data:
    data.append({
        "tweet_id": tweet.id,
        "user_id": tweet.author_id,
        "text": tweet.text
    })

df = pd.DataFrame(data)
print("✅ Collected", len(df), "tweets")
df.head()


TooManyRequests: 429 Too Many Requests
Usage cap exceeded: Monthly product cap

In [ ]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r"http\S+", "", text)          # remove URLs
    text = re.sub(r"@\w+", "", text)             # remove mentions
    text = re.sub(r"#\w+", "", text)             # remove hashtags
    text = re.sub(r"[^A-Za-z\s]", "", text)      # remove special chars/numbers
    text = text.lower()                          # lowercase
    tokens = [w for w in text.split() if w not in stop_words]  # remove stopwords
    return " ".join(tokens)

# Apply cleaning
df = df_raw.copy()
df["cleaned_text"] = df["text"].apply(clean_text)

# ========================
#  Data Quality Checks
# ========================
report = {
    "total_rows": len(df),
    "missing_texts": df["text"].isna().sum(),
    "missing_users": df["user_id"].isna().sum(),
    "duplicate_tweets": df["tweet_id"].duplicated().sum()
}
print("🔍 Data Quality Report:", report)

# ========================
# Save CSV
# ========================
df.to_csv("tweets_cleaned.csv", index=False, encoding="utf-8")
print("✅ Saved cleaned tweets to tweets_cleaned.csv")

from google.colab import files
files.download("tweets_cleaned.csv")


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog